## Feedforward Multilayer Neural Network

The objective of this section is to implement the backpropagation algorithm (SGD with the chain rule) on a single layer neural network using the sigmoid activation function.

Some initial imports:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn import preprocessing

import numpy as np

digits = load_digits()

def one_hot(n_classes, y):
    return np.eye(n_classes)[y]


data = np.asarray(digits.data, dtype='float32')
target = np.asarray(digits.target, dtype='int32')


X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.15, random_state=37)

# Normalise the images
# YOUR CODE HERE
raise NotImplementedError()

Y_test = one_hot(10, y_test)
Y_train = one_hot(10, y_train)

- Implement the `sigmoid` and its element-wise derivative `dsigmoid` functions:

$$
sigmoid(x) = \frac{1}{1 + e^{-x}}
$$

$$
dsigmoid(x) = sigmoid(x) \cdot (1 - sigmoid(x))
$$

Plot both the sigmoid and its derivative and compare it to wikipedia or wolframalpha.

In [2]:
def sigmoid(X):
    # TODO: implement the sigmoid function
    # YOUR CODE HERE
    raise NotImplementedError()


def dsigmoid(X):
    # TODO: derivative of sigmoid
    # YOUR CODE HERE
    raise NotImplementedError()

# YOUR CODE HERE
raise NotImplementedError()

- Implement `forward` and `forward_keep_all` functions for a model with a hidden layer, similar to what we discussed in the slides:
  - $h = sigmoid(\mathbf{W}^h x + b^h)$
  - $y = softmax(\mathbf{W}^o h + b^o)$

Notes: 
  - try to keep the code as similar as possible as the previous notebooks;
  - `forward_keep_activations()` is similar to forward, but also returns hidden activations and pre activations;
- implement the `train` and `loss` functions.

Bonus: reimplementing all from scratch without looking at the solution of the `LogisticRegression` is an excellent exercise.

In [3]:
# some useful functions from before
EPSILON = 1e-8


def nll(Y_true, Y_pred):
    Y_true, Y_pred = np.atleast_2d(Y_true), np.atleast_2d(Y_pred)
    loglikelihoods = np.sum(np.log(EPSILON + Y_pred) * Y_true, axis=1)
    return -np.mean(loglikelihoods)

def softmax(X):
    exp = np.exp(X)
    return exp / np.sum(exp, axis=-1, keepdims=True)

In [4]:
class NeuralNet():
    """MLP with 1 hidden layer with a sigmoid activation"""

    def __init__(self, input_size, hidden_size, output_size):
        self.W_h = np.random.uniform(
            size=(input_size, hidden_size), high=0.01, low=-0.01)
        self.b_h = np.zeros(hidden_size)
        # Initialise self.W_o and self.b_o the weights used
        # to connect the hidden layer to the output
        # YOUR CODE HERE
        raise NotImplementedError()
        self.output_size = output_size
        
    def forward(self, X):
        # TODO: implement the forward step
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def forward_keep_activations(self, X):
        # TODO: compute the forward step but return intermediate results as well
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def loss(self, X, y):
        # TODO: compute the loss of samples X given true labels y
        # YOUR CODE HERE
        raise NotImplementedError()

    def grad_loss(self, x, y_true):
        # TODO: compute the gradient of sample x with respect to the two Ws and bs
        y, h, z_h = self.forward_keep_activations(x)
        grad_z_o = y - one_hot(self.output_size, y_true)
        
        grad_W_o = np.outer(h, grad_z_o)
        grad_b_o = grad_z_o
        grad_h = np.dot(grad_z_o, np.transpose(self.W_o))
        grad_z_h = grad_h * dsigmoid(z_h)
        grad_W_h = np.outer(x, grad_z_h)
        grad_b_h = grad_z_h
        grads = {"W_h": grad_W_h, "b_h": grad_b_h,
                 "W_o": grad_W_o, "b_o": grad_b_o}
        return grads

    def train(self, x, y, learning_rate):
        # YOUR CODE HERE
        raise NotImplementedError()

    def predict(self, X):
        if len(X.shape) == 1:
            return np.argmax(self.forward(X))
        else:
            return np.argmax(self.forward(X), axis=1)

    def accuracy(self, X, y):
        y_preds = np.argmax(self.forward(X), axis=1)
        return np.mean(y_preds == y)


In [5]:
n_features = X_train.shape[1]
n_classes = Y_train.shape[1]

n_hidden = 10
model = NeuralNet(n_features, n_hidden, n_classes)

What do things look like before training?

In [6]:
model.loss(X_train, y_train)

In [7]:
# what should the accuracy of the untrained
# model be?
# YOUR CODE HERE
raise NotImplementedError()

In [8]:
# Plot the predicted class probabilites
# for one sample. How does it compare
# to your expectations?
# YOUR CODE HERE
raise NotImplementedError()

In [9]:
losses, accuracies, accuracies_test = [], [], []
losses.append(model.loss(X_train, y_train))
accuracies.append(model.accuracy(X_train, y_train))
accuracies_test.append(model.accuracy(X_test, y_test))

print("Random init: train loss: %0.4f, train acc: %0.3f, test acc: %0.3f"
      % (losses[-1], accuracies[-1], accuracies_test[-1]))

for epoch in range(15):
    for i, (x, y) in enumerate(zip(X_train, y_train)):
        model.train(x, y, 0.1)

    losses.append(model.loss(X_train, y_train))
    accuracies.append(model.accuracy(X_train, y_train))
    accuracies_test.append(model.accuracy(X_test, y_test))
    print("Epoch #%d, train loss: %0.4f, train acc: %0.3f, test acc: %0.3f"
          % (epoch + 1, losses[-1], accuracies[-1], accuracies_test[-1]))
    
# rerun the notebook without normalising/scaling the images and performance will be worse

96% testing accuracy! Not bad.

In [10]:
plt.plot(losses)
plt.title("Training loss");

In [11]:
plt.plot(accuracies, label='train')
plt.plot(accuracies_test, label='test')
plt.ylim(0, 1.1)
plt.ylabel("accuracy")
plt.legend(loc='best');

## More please!

Exercises

### Hyper parameters settings

- Experiment with different hyper parameters:
  - learning rate,
  - size of hidden layer,
  - initialization scheme: test with 0 initialization vs uniform,
  - implement other activation functions,
  - implement the support for a second hidden layer.


### Mini-batches

- the current implementations of the `train` and `grad_loss` functions currently only accept a single sample at a time:
    - implement the support for training with a mini-batch of 32 samples at a time instead of one,
    - experiment with different sizes of batches,
    - monitor the norm of the average gradients on the full training set at the end of each epoch.


### Momentum

- Bonus: Implement a optimizer with momentum